# Analyzing.

## 1. Location.

In [1]:
from pymongo import MongoClient
import pandas as pd
import time
import importlib
import geopandas as gpd
import data.cleaning as clean
importlib.reload(clean)
import json
from cartoframes.viz import Map, Layer, popup_element

from getpass import getpass
import os
import requests

from dotenv import load_dotenv
import pymongo



Todo el proceso para descargar la base de datos Empresas de Mongo.

In [2]:
client = MongoClient("localhost:27017")

In [3]:
client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [4]:
client.list_database_names()

['Ironhack', 'admin', 'config', 'local']

In [5]:
db = client["Ironhack"]

In [6]:
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'Ironhack')

In [7]:
c = db.get_collection("companies")

Explore the data base to get the information I want for the company.

In [8]:
# This was to look if the data base from Mongo was well downloaded.
# c.find_one()

In [9]:
# This was to find with a specific condition of the ategory, and look for what i wanted to work on.
# list(c.find({}, {"category_code":1}))

#### Will choose the music industry for my company.
Getting the music buissnes companies and start analizing them.

In [10]:
# list of all the companies with category of music in all the world.
#list(c.find({"category_code":"music"}, {"name":1, "_id":0, "category_code":1}))

Get the companies Ubication COUNTRY.

In [11]:
def music_countries (category_code, country_code):
    condition1 = {"category_code":category_code}
    projection = {"name":1, "_id":0, "offices.country_code":country_code}
    countries = (list(c.find(condition1, 
               projection)))
    
    listcountry = [countries[i]["offices"][j]["country_code"] for i in range(len(countries)) for j in range(len(countries[i]["offices"]))]
    return listcountry
    

In [12]:
# list of the countries with the music company industry.
lstcountry = music_countries ("music", 1)

In [13]:
#countries[0]
#countries[0]["offices"]
#countries[0]["offices"][0]["country_code"]

In [14]:
# change it to a dictionary
country = {"country": lstcountry}

In [15]:
# make it a data frame
dfcountry = pd.DataFrame(country)

We see that for much USA is the place for a music company, I wanted to see if I could go to another country that had some type of competition between the USA, but if the company wants to grow, need to be in the USA.

In [16]:
dfcountry.value_counts()[:10]

country
USA        30
GBR         5
FRA         4
DEU         2
BEL         1
BRA         1
ESP         1
ITA         1
LUX         1
NLD         1
dtype: int64

hacer otra columna del value count de los paises

In [17]:
dfcountry.sample(3)

,country
40,FRA
0,USA
37,USA


### Searching for the city

In [18]:
condition1 = {"category_code":"music"}
condition2 = {"offices.city":"San Diego"}
projection = {"name":1, "_id":0, "offices.address1":1}
x = (list(c.find({"$and":
                [condition1, condition2]},
                   projection)))

  

In [19]:
x

[{'name': 'Slacker',
  'offices': [{'address1': '16935 W. Bernardo Dr. Suite 101'}]},
 {'name': 'Bandsintown',
  'offices': [{'address1': '800 W Ivy Street, Unit C'},
   {'address1': '215 Lexington Ave'}]}]

In [20]:
def music_cities (category_code, cities):
    condition1 = {"category_code":category_code}
    projection = {"name":1, "_id":0, "offices.city":cities}
    city = (list(c.find(condition1, 
               projection)))
    
    lstcity = [city[i]["offices"][j]["city"] for i in range(len(city)) for j in range(len(city[i]["offices"]))]
    return lstcity

In [21]:
lstcity = music_cities ("music", 1)

Merge the 2 columns of country and city and new count column

In [22]:
def combine_lists (lst1, lst2):
    # change the 2 lists into a dictionary
    location = {"cities": lst1,
            "country": lst2} 
    # create them a Data Frame.
    location = pd.DataFrame(location)
    # create a new column with the value_counts of cities.
    location["count"] = location.groupby(["cities"])["country"].transform("count")
    return location

In [23]:
location_df = combine_lists (lstcity, lstcountry)

In [24]:
location_df.sample(3)

,cities,country,count
38,San Francisco,USA,9
16,London,GBR,4
39,Brussels,BEL,1


Will do a drop duplicated and drop na 

In [25]:
def clean_df_citycountry (df):
    x = df.combine_first(df).reset_index().reindex(df.columns, axis=1).drop_duplicates()
    x.dropna(how="any", inplace=True)
    return x

In [26]:
location_clean = clean_df_citycountry (location_df)
location_clean.sample(3)

,cities,country,count
44,Sainte Florence,FRA,1
11,Oxford,GBR,1
34,Sao Paulo,BRA,1


Export the data frame of location into a CSV file.

In [27]:
def export_location (df):
    return df.to_csv(f"D:\\ironhack\\proyectos\\GeoSpatialData_proy3\csv\\location_clean.csv")
    

In [28]:
export_location (location_clean)

## 2. Startup environment.

In [29]:
def startups_cooord (city, amount):
    # Conditions for city and amount of the company.
    condition1 = {"offices.city":city}
    condition2 = {"acquisitions.price_amount":{"$gte":amount}}
    projection = {"name":1, "_id":0, "category_code":1, "offices.latitude":1, "offices.longitude":1}
    x = (list(c.find({"$and":
                             [condition1, condition2]},
                   projection)))
    return x

In [30]:
startups = startups_cooord ("New York", 1000000)
startups

[{'name': 'Facebook',
  'category_code': 'social',
  'offices': [{'latitude': 37.41605, 'longitude': -122.151801},
   {'latitude': 53.344104, 'longitude': -6.267494},
   {'latitude': 40.7557162, 'longitude': -73.9792469}]},
 {'name': 'Viacom',
  'category_code': 'web',
  'offices': [{'latitude': 40.757725, 'longitude': -73.986011}]},
 {'name': 'Google',
  'category_code': 'search',
  'offices': [{'latitude': 37.421972, 'longitude': -122.084143},
   {'latitude': 42.280988, 'longitude': -83.748882},
   {'latitude': 33.781466, 'longitude': -84.387519},
   {'latitude': 30.351416, 'longitude': -97.751382},
   {'latitude': 40.021416, 'longitude': -105.260931},
   {'latitude': 42.36305, 'longitude': -71.087478},
   {'latitude': 41.889474, 'longitude': -87.628912},
   {'latitude': 32.981465, 'longitude': -97.018196},
   {'latitude': 32.809161, 'longitude': -96.807693},
   {'latitude': 39.62919, 'longitude': -104.898419},
   {'latitude': 42.547807, 'longitude': -83.215498},
   {'latitude': 33.6

In [31]:
def lat_startup (dict):
    x = []
    for i in range(len(startups)):
        x.append(startups[i]["offices"][0]["latitude"])
    return x

In [32]:
def long_startup (dict):
    y = []
    for i in range(len(startups)):
        y.append(startups[i]["offices"][0]["longitude"])
    return y

In [33]:
startups_lat = lat_startup (startups)

In [34]:
startups_long = long_startup (startups)

In [35]:
# Filter again the startups but without the latitud and logitude and turn it into a data frame.
def startups_name (city, amount):
    # Conditions for city and amount of the company.
    condition1 = {"offices.city":city}
    condition2 = {"acquisitions.price_amount":{"$gte":amount}}
    projection = {"name":1, "_id":0, "category_code":1}
    x = (list(c.find({"$and":
                             [condition1, condition2]},
                   projection)))
    # give a Data frame with conditions and address.
    df = pd.DataFrame(x)
    return df    

In [36]:
start_names = startups_name ("New York", 1000000)
start_names.sample(3)

,name,category_code
33,Internap,web
1,Viacom,web
14,Comverse Technology,software


Make a dictionary with all values to create a data base.

In [37]:
# dictionary for latitud and long, to create df.
def coord_df (lats, longs):
    lon_lat = {"latitud": lats,
              "longitude": longs}
    df = pd.DataFrame(lon_lat)
    return df

In [38]:
df = coord_df (startups_lat, startups_long)

In [39]:
df.sample(3)

,latitud,longitude
34,37.445111,-122.160779
1,40.757725,-73.986011
39,51.452512,-2.590974


In [40]:
# concat the 2 df.
def concat_axis1 (df1, df2):
    starts = pd.concat([df1, df2], axis=1)
    return starts

In [41]:
startups = concat_axis1 (start_names, df)

In [42]:
startups.sample()

,name,category_code,latitud,longitude
24,WPP,public_relations,40.751513,-73.977947


Get to de df new points columns for plot it in the map.

Export clean Data frame of startups, their category and their address inside San diego.
export dictionary for maping.

como guardarlo en json para exportarlo??????????????????????????????

In [43]:
#with open ("startupsNYjson.json", "w") as f:
#    json.dump(startups, f)

In [44]:
def export_NY_startups (df):
    return df.to_csv(f"D:\\ironhack\\proyectos\\GeoSpatialData_proy3\csv\\startups_newyork.csv")

In [45]:
export_NY_startups (startups)

## 3. Foursquare.

In [46]:
token = getpass()

········


### 3.1. Starbucks.

In [56]:
url = "https://api.foursquare.com/v3/places/search"

params = {
    "query": "starbucks",
    "ll": "40.751693,-73.989676",
    "open_now": "true",
    "sort":"DISTANCE"
    
}

headers = {
    "Accept": "application/json",
    "Authorization": token
}

response = requests.request("GET", url, params=params, headers=headers)

response

<Response [200]>

In [57]:
response.json()["results"]

[{'fsq_id': '59a8ce646f0aa256da9a714f',
  'categories': [{'id': 13035,
    'name': 'Coffee Shop',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
     'suffix': '.png'}}],
  'chains': [{'id': 'ab4c54c0-d68a-012e-5619-003048cad9da',
    'name': 'Starbucks'}],
  'distance': 60,
  'geocodes': {'main': {'latitude': 40.751907, 'longitude': -73.990217},
   'roof': {'latitude': 40.751907, 'longitude': -73.990217}},
  'link': '/v3/places/59a8ce646f0aa256da9a714f',
  'location': {'address': '462 7th Ave',
   'address_extended': 'Frnt 3',
   'census_block': '360610109001004',
   'country': 'US',
   'cross_street': 'W 35th St',
   'dma': 'New York',
   'formatted_address': '462 7th Ave (W 35th St), New York, NY 10018',
   'locality': 'New York',
   'postcode': '10018',
   'region': 'NY'},
  'name': 'Starbucks',
  'related_places': {},
  'timezone': 'America/New_York'},
 {'fsq_id': '61ed943409f1455a665c6353',
  'categories': [{'id': 13035,
    'name': 'Coffee Shop

In [58]:
coordinates = response.json()["results"]

In [59]:
len(coordinates)

10

In [51]:
response.json()["results"][0]["geocodes"]["main"]["latitude"]

40.751907

In [52]:
# Make a dictionary for a single index.
def starbucks (dict_):
    dicts = {"name": dict_["name"],
            "latitude": dict_["geocodes"]["main"]["latitude"],
             "longitude": dict_["geocodes"]["main"]["longitude"]}
    return dicts

In [53]:
starbucks (response.json()["results"][0])

{'name': 'Starbucks', 'latitude': 40.751907, 'longitude': -73.990217}

In [54]:
# Make a dictionary with the dictionary of the single index but iteration with the rest.
def starbucks_complete (dicc):
    lst = []
    for i in response.json()["results"]:
        lst.append(starbucks(i))
    # Make the dictionary a data frame
    return lst

In [55]:
starbucks = pd.DataFrame(lst)

NameError: name 'lst' is not defined

Export data frame.

In [ ]:
def export_NY_starbucks (df):
    return df.to_csv(f"D:\\ironhack\\proyectos\\GeoSpatialData_proy3\csv\\starbucks_newyork.csv")

In [ ]:
export_NY_starbucks (starbucks)